In [ ]:
%matplotlib inline

import cv2
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from imutils.object_detection import non_max_suppression


def plot_image(image):    
    """
    Plot a grayscale image in the original size - takes time
    """
    fig = plt.figure(figsize=(40, 40))
    ax1 = plt.subplot(1, 1, 1) 
    ax1.imshow(image, cmap='gray')  


In [ ]:
from pathlib import Path

# Load the main image and template
image_path = Path('E:/DATA/AI4Life_Pr26/20240805_Trial_data_fiducial_particles/240723_JB294_CLEM-AI4life_sample1/pos1/240726_JB295_HEK293_CLEM_LAMP1-488_Particles-555_grid4_pos1_bin4_EM.tif')
print(image_path.exists())

template_path = Path('E:/DATA/AI4Life_Pr26/20240805_Trial_data_fiducial_particles/240723_JB294_CLEM-AI4life_sample1/pos1/240726_JB295_HEK293_CLEM_LAMP1-488_Particles-555_grid4_pos1_bin4_EM_template.tif')
print(template_path.exists())

image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)

# Parameters
matching_threshold = 0.68  # threshold for template matching
overlap_threshold =0.1

# Get template dimensions
h, w = template.shape

In [ ]:
# Display the input image
plot_image(image)

In [ ]:
def non_max_suppression(boxes, scores, threshold):
    # Sort boxes by score in descending order
    sorted_indices = np.argsort(scores)[::-1]
    print("Indices: ", sorted_indices)
    
    keep_boxes = []
    
    while sorted_indices.size > 0:
        # Pick the box with the highest score
        box_id = sorted_indices[0]
        keep_boxes.append(box_id)
        
        # Calculate IoU of the picked box with the rest
        ious = calculate_iou(boxes[box_id], boxes[sorted_indices[1:]])
        
        # Remove boxes with IoU over the threshold
        keep_indices = np.where(ious < threshold)[0]
        
        # Update the indices
        sorted_indices = sorted_indices[keep_indices + 1]
    print("Keep boxes: ", keep_boxes)
    return keep_boxes

def calculate_iou(box, boxes):
    # Calculate intersection areas
    x1 = np.maximum(box[0], boxes[:, 0])
    y1 = np.maximum(box[1], boxes[:, 1])
    x2 = np.minimum(box[2], boxes[:, 2])
    y2 = np.minimum(box[3], boxes[:, 3])
    
    intersection_area = np.maximum(0, x2 - x1) * np.maximum(0, y2 - y1)
    
    # Calculate union areas
    box_area = (box[2] - box[0]) * (box[3] - box[1])
    boxes_area = (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1])
    union_area = box_area + boxes_area - intersection_area
    
    # Calculate IoU
    iou = intersection_area / union_area
    return iou

def template_matching(image, template, threshold):
    result = cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)
    #plot_image(result)
    locations = np.where(result >= threshold)
    scores = result[locations]
    matches = list(zip(*locations[::-1]))
    
    return matches, scores

In [ ]:
# Perform template matching
matches,scores = template_matching(image, template, matching_threshold)

#print("Scores:", scores)
#print("Match locations: ", matches)

# Draw rectangles around the matched regions
#img = image
#for pt in matches:
#    cv2.rectangle(img, pt, (pt[0] + w, pt[1] + h), (0, 255, 0), 2)
    
#plot_image(img)

# Create bounding boxes
w, h = template.shape[::-1]
boxes = [(x, y, x + w, y + h) for (x, y) in matches]
#print("Boxes: ", boxes)

# Apply non-maximum suppression for filtering out overlapping boxes
keep_ids = non_max_suppression(np.array(boxes), scores, overlap_threshold)
print("Kept_boxes: ", np.array(keep_ids))
#print(matches[keep_ids[0]])

loc = [matches[keep_id] for keep_id in keep_ids]
print(loc)
# Draw rectangles around the matched regions
img2 = image
for pt in loc:
    cv2.rectangle(img2, pt, (pt[0] + w, pt[1] + h), (0, 255, 0), 2)

In [ ]:
from matplotlib import pyplot as plt
plot_image(img2)
plt.savefig('fiducial_detection.png')